In [1]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# --- 설정 (Configuration) ---
MINIO_ENDPOINT = "http://titan-minio:9000"
ACCESS_KEY = "minioadmin"
SECRET_KEY = "minioadmin"

spark = SparkSession.builder \
    .appName("TitanLog-Optimization") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.533") \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

# 1. 원본 데이터 로딩 (Silver)
df = spark.read.parquet("s3a://silver/log_v1")
print(f"전체 데이터 수: {df.count()}건")

# ==========================================
# 기술 1: 파티셔닝 (Partitioning)
# 데이터를 '이벤트 타입(view, cart, purchase)'별로 폴더를 나눠서 저장합니다.
# ==========================================
print("\n--- [1] 파티셔닝 저장 시작 ---")
# event_type 컬럼을 기준으로 물리적인 폴더를 나눕니다.
df.write.mode("overwrite").partitionBy("event_type").parquet("s3a://silver/log_partitioned")
print("파티셔닝 저장 완료! MinIO에서 'silver/log_partitioned' 폴더를 확인해보세요.")

# 비교 실험: "구매(purchase)" 데이터만 찾을 때 속도 차이
# (데이터가 적어서 차이가 미미할 수 있지만, 원리는 중요합니다!)

# Case A: 그냥 통짜 파일에서 찾기 (Full Scan)
start = time.time()
df.filter(col("event_type") == "purchase").count()
print(f"일반 검색 소요 시간: {time.time() - start:.4f}초")

# Case B: 파티셔닝된 폴더에서 찾기 (Partition Pruning)
# Spark가 'event_type=purchase' 폴더만 쏙 골라서 읽습니다.
partitioned_df = spark.read.parquet("s3a://silver/log_partitioned")
start = time.time()
partitioned_df.filter(col("event_type") == "purchase").count()
print(f"파티셔닝 검색 소요 시간: {time.time() - start:.4f}초")


# ==========================================
# 기술 2: 캐싱 (Caching)
# 데이터를 메모리(RAM)에 고정시켜서 두 번째 실행부터 광속으로 만듭니다.
# ==========================================
print("\n--- [2] 캐싱 실험 시작 ---")

# 무거운 작업 시뮬레이션 (그룹핑 집계)
heavy_df = df.groupBy("user_id").count()

# 첫 번째 실행 (디스크에서 읽느라 느림)
start = time.time()
print(f"1차 실행 결과: {heavy_df.count()}명 (캐시 X)")
print(f"1차 소요 시간: {time.time() - start:.4f}초")

# 캐시 등록! (메모리에 올려라!)
heavy_df.cache()

# 두 번째 실행 (메모리에서 바로 꺼냄 -> 개빠름)
start = time.time()
print(f"2차 실행 결과: {heavy_df.count()}명 (캐시 O)")
print(f"2차 소요 시간: {time.time() - start:.4f}초")

# (중요) 메모리 비워주기
heavy_df.unpersist()
print("캐시 메모리 해제 완료")

전체 데이터 수: 5000건

--- [1] 파티셔닝 저장 시작 ---
파티셔닝 저장 완료! MinIO에서 'silver/log_partitioned' 폴더를 확인해보세요.
일반 검색 소요 시간: 0.1467초
파티셔닝 검색 소요 시간: 0.0976초

--- [2] 캐싱 실험 시작 ---
1차 실행 결과: 101명 (캐시 X)
⏱1차 소요 시간: 0.2947초
2차 실행 결과: 101명 (캐시 O)
⚡ 2차 소요 시간: 0.2056초
캐시 메모리 해제 완료
